In [ ]:
import pandas as pd

### 예제 문제 
1. csv 폴더 안에 있는 tran_1, tran_2 파일 로드 
2. 두개의 데이터프레임을 단순하게 행을 추가하는 결합
3. csv 폴더 안에 있는 tran_d_1, tran_d_2 파일 로드 
4. 두개의 데이터프레임을 단순하게 행을 추가하는 결합 
5. 2번 과정에서 나온 데이터프레임과 4번 과정에서 나온 데이터프레임을 특정한 조건에 맞춰서 열을 결합 기준은 2번 데이터프레임을 기준으로 결합

In [ ]:
# tran_1, tran_2 로드해서 유니언 결합
tran_1 = pd.read_csv("../../csv/tran_1.csv")
tran_2 = pd.read_csv("../../csv/tran_2.csv")
print(tran_1.head(1))
print(tran_2.head(1))

In [ ]:
tran = pd.concat([tran_1, tran_2], axis=0, ignore_index=True)

In [ ]:
tran_d_1 = pd.read_csv('../../csv/tran_d_1.csv')
tran_d_2 = pd.read_csv('../../csv/tran_d_2.csv')
print(tran_d_1.head(1))
print(tran_d_2.head(1))

In [ ]:
tran_d = pd.concat([tran_d_1, tran_d_2], axis=0, ignore_index=True)

In [ ]:
print(tran.head(1))
print(tran_d.head(1))

In [ ]:
# tran, tran_d 두개의 데이터프레임을 조인결합 
tran = pd.merge(tran, tran_d, on = 'transaction_id', how = 'inner')

In [ ]:
tran_d['transaction_id'].value_counts()

1. csv폴더에서 customer_master, item_master 파일 로드 
2. 4개의 데이터프레임이 결합되어 있는 tran 데이터프레임에 조인 결합

In [ ]:
customer = pd.read_csv("../../csv/customer_master.csv")
customer.head(1)

In [ ]:
tran.head(1)

In [ ]:
tran = pd.merge(tran, customer, on='customer_id', how='left')

In [ ]:
item = pd.read_csv('../../csv/item_master.csv')
item.head(1)

In [ ]:
tran.head(1)

In [22]:
df = pd.merge(tran, item, on='item_id', how='left')

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7144 entries, 0 to 7143
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   transaction_id  7144 non-null   object 
 1   price           7144 non-null   int64  
 2   payment_date    7144 non-null   object 
 3   customer_id     7144 non-null   object 
 4   detail_id       7144 non-null   int64  
 5   item_id         7144 non-null   object 
 6   quantity        7144 non-null   int64  
 7   name            6069 non-null   object 
 8   class           6069 non-null   object 
 9   gender          6069 non-null   object 
 10  start_date      6069 non-null   object 
 11  end_date        1978 non-null   object 
 12  campaign_id     6069 non-null   object 
 13  is_deleted      6069 non-null   float64
 14  item_name       7144 non-null   object 
 15  item_price      7144 non-null   int64  
dtypes: float64(1), int64(4), object(11)
memory usage: 948.8+ KB


In [ ]:
df[['price', 'quantity', 'item_price']]

1. 거래금액이 가장 큰 데이터는 무엇인가?
2. 거래가 가장 많이 된 물품의 상위 1개를 출력

In [26]:
df['price'].max()

750000

In [ ]:
# price를 기준으로 내림차순 -> 상위 5개 
df.sort_values('price', ascending=False).head(5)

In [29]:
## 거래가 가장 많이 된 물품을 확인 
# 1. 아이템을 기준으로 그룹화 -> quantity의 합계
group_data = df[['item_id', 'quantity']].groupby('item_id').sum()
# group_data에서 quantity를 기준으로 내림차순 정렬 -> 상위 3개
group_data.sort_values('quantity', ascending=False).head(3)

,quantity
item_id,
S001,3043
S005,1822
S002,1785
